# LangChain 核心模块学习：Chains

对于简单的大模型应用，单独使用语言模型（LLMs）是可以的。

**但更复杂的大模型应用需要将 `LLMs` 和 `Chat Models` 链接在一起 - 要么彼此链接，要么与其他组件链接。**

LangChain 为这种“链式”应用程序提供了 `Chain` 接口。

LangChain 以通用方式定义了 `Chain`，它是对组件进行调用序列的集合，其中可以包含其他链。

## LLMChain

LLMChain 是 LangChain 中最简单的链，作为其他复杂 Chains 和 Agents 的内部调用，被广泛应用。

一个LLMChain由PromptTemplate和语言模型（LLM or Chat Model）组成。它使用直接传入（或 memory 提供）的 key-value 来规范化生成 Prompt Template（提示模板），并将生成的 prompt （格式化后的字符串）传递给大模型，并返回大模型输出。

![](../images/llm_chain.png)

## Router Chain: 实现条件判断的大模型调用


这段代码构建了一个可定制的链路系统，用户可以提供不同的输入提示，并根据这些提示获取适当的响应。

主要逻辑：从`prompt_infos`创建多个`LLMChain`对象，并将它们保存在一个字典中，然后创建一个默认的`ConversationChain`，最后创建一个带有路由功能的`MultiPromptChain`。

![](../images/router_chain.png)

In [5]:
from langchain.chains.router import MultiPromptChain
from langchain_openai import OpenAI
from langchain.chains.conversation.base import ConversationChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

In [32]:
physics_template = """你是一位非常聪明的物理教授。
你擅长以简洁易懂的方式回答关于物理的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


math_template = """你是一位很棒的数学家。你擅长回答数学问题。
之所以如此出色，是因为你能够将难题分解成各个组成部分，
先回答这些组成部分，然后再将它们整合起来回答更广泛的问题。

这是一个问题：
{input}"""

# 生物老师
biology_template = """你是一位很棒的生物老师。
你擅长以简洁易懂的方式回答关于生物学的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""
# 计算机老师
computer_template = """你是一位很棒的计算机老师。
你擅长以简洁易懂的方式回答关于计算机的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""

# 汉语言文学老师
chinese_template = """你是一位很棒的汉语言文学老师。
你擅长以简洁易懂的方式回答关于汉语言文学的问题。
当你不知道某个问题的答案时，你会坦诚承认。

这是一个问题：
{input}"""


In [33]:
prompt_infos = [
    {
        "name": "物理",
        "description": "适用于回答物理问题",
        "prompt_template": physics_template,
    },
    {
        "name": "数学",
        "description": "适用于回答数学问题",
        "prompt_template": math_template,
    },
    {
        "name": "生物",
        "description": "适用于回答生物问题",
        "prompt_template": biology_template,
    },
    {
        "name": "计算机",
        "description": "适用于回答计算机问题",
        "prompt_template": computer_template,
    },
    {
        "name": "汉语言文学",
        "description": "适用于回答汉语言文学问题",
        "prompt_template": chinese_template,
    },
]

In [34]:
llm = OpenAI(model_name="gpt-3.5-turbo-instruct")

In [35]:
# 创建一个空的目标链字典，用于存放根据prompt_infos生成的LLMChain。
destination_chains = {}

# 遍历prompt_infos列表，为每个信息创建一个LLMChain。
for p_info in prompt_infos:
    name = p_info["name"]  # 提取名称
    prompt_template = p_info["prompt_template"]  # 提取模板
    # 创建PromptTemplate对象
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    # 使用上述模板和llm对象创建LLMChain对象
    chain = LLMChain(llm=llm, prompt=prompt)
    # 将新创建的chain对象添加到destination_chains字典中
    destination_chains[name] = chain

# 创建一个默认的ConversationChain,兜底的，相当于if else语句中的最后一个else
default_chain = ConversationChain(llm=llm, output_key="text")

In [36]:
type(default_chain)

langchain.chains.conversation.base.ConversationChain

### 使用 LLMRouterChain 实现条件判断调用

这段代码定义了一个chain对象（LLMRouterChain），该对象首先使用router_chain来决定哪个destination_chain应该被执行，如果没有合适的目标链，则默认使用default_chain。

In [37]:
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

In [38]:
# 从prompt_infos中提取目标信息并将其转化为字符串列表
destinations = [f"{p['name']}: {p['description']}" for p in prompt_infos]
# 使用join方法将列表转化为字符串，每个元素之间用换行符分隔
destinations_str = "\n".join(destinations)
# 根据MULTI_PROMPT_ROUTER_TEMPLATE格式化字符串和destinations_str创建路由模板
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
# 创建路由的PromptTemplate
router_prompt = PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser(),
)
# 使用上述路由模板和llm对象创建LLMRouterChain对象
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [39]:
print(destinations)

['物理: 适用于回答物理问题', '数学: 适用于回答数学问题', '生物: 适用于回答生物问题', '计算机: 适用于回答计算机问题', '汉语言文学: 适用于回答汉语言文学问题']


In [40]:
print(destinations_str)

物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题


In [41]:
# 每次format都会去掉模板中的变量的n/2个花括号，其中n是变量的花括号数据量（n>1）
"测试：{{{{china}}}} {hello}, {{book}}".format(hello="hello")

'测试：{{china}} hello, {book}'

In [42]:
print(MULTI_PROMPT_ROUTER_TEMPLATE)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
{destinations}

<< INPUT >>
{{input}}

<< OUTPUT (must include ```json at the start of the respon

In [43]:
print(router_template)

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
物理: 适用于回答物理问题
数学: 适用于回答数学问题
生物: 适用于回答生物问题
计算机: 适用于回答计算机问题
汉语言文学: 适用于回答汉语言文学问题

<< INPUT >>
{input}

<

In [44]:
# 创建MultiPromptChain对象，其中包含了路由链，目标链和默认链。
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=default_chain,
    verbose=True,
)

In [47]:
router_chain.verbose = True

In [45]:
print(chain.run("黑体辐射是什么？?"))



> Entering new MultiPromptChain chain...
物理: {'input': '黑体辐射是什么？?'}
> Finished chain.


黑体辐射是一种理想的物理现象，它指的是处于热平衡状态、不透光且完全吸收所有进入的辐射的物体所发射出的辐射。根据普朗克定律，黑体辐射的光谱分布与温度有关，即温度越高，发射的光谱分布也会更强。这个现象对于理解热辐射和能量传输有重要意义。


In [46]:
print(
    chain.run(
        "大于40的第一个质数是多少，使得这个质数加一能被3整除？"
    )
)



> Entering new MultiPromptChain chain...
数学: {'input': '找到第一个大于40的质数，使得这个质数加一能被3整除。'}
> Finished chain.


首先，我们需要了解质数的定义。质数是指只能被1和自身整除的数，也就是除了1和它本身之外没有其他因数的数。

根据题目要求，我们需要找到一个大于40的质数，所以我们可以从41开始检查数是否为质数。

首先，我们可以将41除以2，发现余数为1，所以41不是偶数，也就是说41不可能被2整除，因此41可能是一个质数。

然后，我们可以继续将41除以3，发现余数为2，所以41也不可能被3整除，因此41也不是一个能被3整除的数。

接着，我们可以将41除以5，发现余数为1，所以41也不可能被5整除，因此41也不是一个能被5整除的数。

继续按照这个方法，我们可以将41分别除以7


In [29]:
print(chain.run("黑洞是什么？"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
物理: {'input': '黑洞是什么？'}
> Finished chain.


黑洞是宇宙中非常密集的物质聚集体，其引力非常强大，甚至连光也无法逃离它的吸引。它的引力是由其质量和体积决定的，通常被描述为一个密度极高、质量极大的奇点。我们无法直接观测黑洞，但我们可以通过它们产生的引力效应来研究它们。黑洞是一种非常有趣的天体，也是物理学中重要的研究对象。


In [48]:
print(chain.run("硬盘的分类有哪些?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
计算机: {'input': '硬盘的分类有哪些?'}
> Finished chain.


硬盘可以按照存储介质、接口类型、转速等多种方式进行分类。最常见的分类方式有以下几类：
1. 根据存储介质：固态硬盘（SSD）和机械硬盘（HDD）两类。
2. 根据接口类型：SATA、SAS、SCSI等。
3. 根据转速：7200转、10000转、15000转等。
4. 根据外形尺寸：2.5英寸、3.5英寸等。
5. 根据用途：普通硬盘、服务器硬盘、笔记本硬盘、移动硬盘等。


In [49]:
print(chain.run("人的祖先是什么?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
生物: {'input': '人的祖先'}
> Finished chain.
是什么？

这是一个很复杂的问题，因为人的祖先可以追溯到几百万年前的灵长类动物。根据科学研究，最早的人类祖先可以追溯到大约600万年前的非洲地区。这些祖先是类人猿，与今天的黑猩猩和大猩猩有着共同的祖先。随着进化的过程，这些类人猿逐渐发展成为早期的人类，包括直立人和尼安德特人。最后，大约20万年前，现代人类（智人）出现并逐渐取代其他早期人类。所以，人类的祖先可以说是非洲的类人猿。但是，这只是一个简单的概括，实际情况可能更复杂，科


In [50]:
print(chain.run("李白最出名的诗有哪些?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
汉语言文学: {'input': '李白最出名的诗有哪些?'}
> Finished chain.


李白最出名的诗包括《将进酒》、《静夜思》、《早发白帝城》、《望庐山瀑布》等。这些诗作都是他的代表作，被广泛传诵和赞美。不过，李白的诗作数量众多，每个人认为最出名的诗可能会有所不同。


In [54]:
print(chain.run("红烧肉的做法?"))



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
None: {'input': '红烧肉的做法?'}
> Finished chain.
 红烧肉是一道中国菜肴，主要由猪肉（通常是五花肉）和配料（如酱油、糖、料酒、姜等）烧制而成。首先将猪肉切成小块，然后用冷水煮沸，撇去浮沫。接着将肉块捞出，用冷水冲洗干净，备用。将炒锅烧热，放入适量的油，然后加入葱姜蒜炒香。接着放入肉块煸炒，直到肉块变色。然后加入适量的酱油、糖和料酒，翻炒均匀。接着加入适量的水，盖上盖子，用小火煨烧约


### Homework

#### （已完成）扩展 Demo：实现生物、计算机和汉语言文学老师 PromptTemplates 及对应 Chains